In [1]:
import pandas as pd
import numpy as n
import math

In [2]:
config = {
  "worker" : {
    "min_storage_gb": 50,
    "extra": 3,
    "reserved_cpu": 2,
    "reserved_ram_gb": 4,
    "spare_resource_pct": 20
  }
}

In [3]:
pods = pd.read_csv(r"pods.csv")
vms = pd.read_csv(r"vms.csv")

In [4]:
for workload in [pods, vms]:
  workload['total_vcpu'] = workload['count'] * workload['vcpu']
  workload['total_ram_gb'] = workload['count'] * workload['ram_gb']
  workload['total_storage_gb'] = workload['count'] * (workload['estorage_gb'] + workload['pstorage_gb'])
  workload


# Pods

In [5]:
pods

,site,domain,env,host_profile,name,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
0,Site 1,Non-Prod,Dev,K8s-Worker-Apps,ACME API,1,1,4,2,20,1,4,22
1,Site 1,Non-Prod,Dev,K8s-Worker-Apps,ACME Backend API,1,1,4,2,20,1,4,22
2,Site 1,Non-Prod,Dev,K8s-Worker-Apps,Foo Account API,1,1,2,2,20,1,2,22
3,Site 1,Non-Prod,Dev,K8s-Worker-Apps,Foo Kong API,1,1,2,2,20,1,2,22
4,Site 1,Non-Prod,Dev,K8s-Worker-Apps,Foo SSO API,1,1,2,2,20,1,2,22
5,Site 1,Non-Prod,Dev,K8s-Worker-Apps,SMS Gateway,1,2,8,2,20,2,8,22
6,Site 1,Non-Prod,Dev,K8s-Worker-DB1,ACME Database,1,2,8,2,20,2,8,22
7,Site 1,Non-Prod,Dev,K8s-Worker-DB2,Foo Database,1,2,8,2,20,2,8,22
8,Site 1,Non-Prod,Dev,K8s-Worker-DS,ACME Eventbus,1,1,4,2,20,1,4,22
9,Site 1,Non-Prod,Dev,K8s-Worker-DS,Foo Elasticsearch,1,1,2,2,20,1,2,22


# Static Virtual Machines

In [6]:
vms

,site,domain,env,host_profile,name,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
0,Site 2,DR,Management,Other,Other,15,6,18,0,256,90,270,3840
1,Site 2,DR,Management,NSX,NSX Controller,8,8,32,0,200,64,256,1600
2,Site 2,DR,Infra,vRPA,vRPA Addtl,2,8,32,0,35,16,64,70
3,Site 2,DR,Infra,NSX,NSX-Edge,2,16,64,50,200,32,128,500
4,Site 2,DR,DR,K8s-Master,K8s-Master,3,4,16,2,200,12,48,606
5,Site 2,DR,DR-Reporting,Reporting Server,Reporting Server,1,8,64,20,600,8,64,620
6,Site 1,Prod,Prod,K8s-Master,K8s-Master,3,4,16,2,50,12,48,156
7,Site 1,Prod,Prod-Reporting,Reporting Server,Reporting Server,1,8,64,20,600,8,64,620
8,Site 1,Prod,Infra,vRPA,vRPA Addtl,2,8,32,0,35,16,64,70
9,Site 1,Prod,Infra,NSX,NSX-Edge,2,16,64,50,200,32,128,500


In [7]:
pod_summary = pods.groupby(["site", "domain", "env", "host_profile"])
workers = []
worker_spare_resource = 1 + config["worker"]["spare_resource_pct"]/100
for name, group in pod_summary:
   site = name[0]
   domain = name[1]
   env = name[2]
   host_profile = name[3]
   worker_name = name[3]
   
   
   pod_group = group.sum()   

   pods_per_worker = math.floor(math.sqrt(pod_group['count']))
  #  worker_name = worker_name + " PPW(" + str(pods_per_worker) + ")"

   min_workers = math.ceil(pod_group['count']/pods_per_worker)
   
   # 2 vcpu for system service, 1.2 time (or 20% extra) for spare capacity
   workload_cpu = pod_group['total_vcpu']/min_workers
   min_worker_cpu = math.ceil( ( workload_cpu + config["worker"]["reserved_cpu"] ) * worker_spare_resource)
   
   # 4 gb for system service, 1.2 time (or 20% extra) for spare capacity
   workload_ram = pod_group['total_ram_gb']/min_workers
   min_worker_ram = math.ceil( ( workload_ram + config["worker"]["reserved_ram_gb"] ) * worker_spare_resource )

   min_worker_estorage = config["worker"]["min_storage_gb"]
   min_worker_pstorage = pod_group['total_storage_gb']/min_workers
   
   worker_count = min_workers + config["worker"]["extra"]
   worker_vcpu = min_worker_cpu
   worker_ram_gb = min_worker_ram
   worker_estorage_gb = math.ceil(min_worker_estorage)
   worker_pstorage_gb = math.ceil(min_worker_pstorage)
   total_worker_vcpu = worker_count * worker_vcpu
   total_worker_ram_gb = worker_count * worker_ram_gb
   total_worker_storage_gb = worker_count * (worker_estorage_gb + worker_pstorage_gb)

   workers.append([site,domain,env,host_profile, worker_name, worker_count, worker_vcpu, worker_ram_gb, worker_estorage_gb, worker_pstorage_gb, total_worker_vcpu, total_worker_ram_gb, total_worker_storage_gb])


# Kubernetes Workers

In [8]:
workers_df = pd.DataFrame(data=workers, columns=vms.columns)
workers_df

,site,domain,env,host_profile,name,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
0,Site 1,Non-Prod,Dev,K8s-Worker-Apps,K8s-Worker-Apps,6,6,14,50,44,36,84,564
1,Site 1,Non-Prod,Dev,K8s-Worker-DB1,K8s-Worker-DB1,4,5,15,50,22,20,60,288
2,Site 1,Non-Prod,Dev,K8s-Worker-DB2,K8s-Worker-DB2,4,5,15,50,22,20,60,288
3,Site 1,Non-Prod,Dev,K8s-Worker-DS,K8s-Worker-DS,5,6,18,50,44,30,90,470
4,Site 1,Non-Prod,Pre-Prod,K8s-Worker-Apps,K8s-Worker-Apps,7,11,32,50,66,77,224,812
5,Site 1,Non-Prod,Pre-Prod,K8s-Worker-DB1,K8s-Worker-DB1,6,5,15,50,22,30,90,432
6,Site 1,Non-Prod,Pre-Prod,K8s-Worker-DB2,K8s-Worker-DB2,6,5,15,50,22,30,90,432
7,Site 1,Non-Prod,Pre-Prod,K8s-Worker-DS,K8s-Worker-DS,7,14,45,50,66,98,315,812
8,Site 1,Non-Prod,Staging,K8s-Worker-Apps,K8s-Worker-Apps,6,8,23,50,44,48,138,564
9,Site 1,Non-Prod,Staging,K8s-Worker-DB1,K8s-Worker-DB1,4,5,15,50,22,20,60,288


# Virtual Infrastructure Detailed

In [9]:
site = vms.append(workers_df).sort_values(["site","domain", "env", "host_profile", "name"])
site

,site,domain,env,host_profile,name,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
18,Site 1,Non-Prod,Dev,K8s-Master,K8s-Master,3,4,16,2,50,12,48,156
0,Site 1,Non-Prod,Dev,K8s-Worker-Apps,K8s-Worker-Apps,6,6,14,50,44,36,84,564
1,Site 1,Non-Prod,Dev,K8s-Worker-DB1,K8s-Worker-DB1,4,5,15,50,22,20,60,288
2,Site 1,Non-Prod,Dev,K8s-Worker-DB2,K8s-Worker-DB2,4,5,15,50,22,20,60,288
3,Site 1,Non-Prod,Dev,K8s-Worker-DS,K8s-Worker-DS,5,6,18,50,44,30,90,470
17,Site 1,Non-Prod,Dev,Reporting Server,Reporting Server,1,2,16,20,20,2,16,40
16,Site 1,Non-Prod,Infra,NSX,NSX-Edge,2,16,64,50,200,32,128,500
15,Site 1,Non-Prod,Management,NSX,NSX Controller,8,8,32,0,200,64,256,1600
14,Site 1,Non-Prod,Management,Other,Other,15,6,18,0,256,90,270,3840
13,Site 1,Non-Prod,Pre-Prod,K8s-Master,K8s-Master,3,4,16,2,50,12,48,156


# Total Virtual Infrastructure Footprint By Site By Environment

In [10]:
site.groupby(["site","domain"]).sum()


count  vcpu  ram_gb  estorage_gb  pstorage_gb  total_vcpu  \
site   domain                                                                
Site 1 Non-Prod    101   133     464          716         1306         707   
       Prod         40   126     496          272         5227         880   
Site 2 DR           63   140     546          272         5833        1034   

                 total_ram_gb  total_storage_gb  
site   domain                                    
Site 1 Non-Prod          2277             12236  
       Prod              3272             39562  
Site 2 DR                3798             45452